## load the packages and data we need

In [ ]:
library('tidyverse')

# load the combined station data
station_data <- read_csv(file.path('data', 'combined_stations.csv'))

## Investigate the relationship between `tmax` and `sun` overall, and by individual seasons, using `cor()`. What kind of relationship do these variables appear to have? Remember to use `drop_na()` to remove missing values!

First, overall. Let's first make a scatter plot showing the values of `tmax` vs `sun`:

In [ ]:
ggplot(data=station_data, mapping=aes(x=sun, y=tmax)) + geom_point()

This looks like an overall positive relationship (and a fairly strong one at that). Now, we can use `summarize()` and `cor()` to calculate the different correlation values for these two variables, making sure to only include observations where we have values of both `tmax` and `sun` (**complete.obs**):

In [ ]:
station_data |> 
    summarize(
        pearson = cor(sun, tmax, use='complete.obs'), # calculate pearson's r for sun and tmax
        spearman = cor(sun, tmax, use='complete.obs', method='spearman'), # calculate spearman's rho for sun and tmax
        kendall = cor(sun, tmax, use='complete.obs', method='kendall') # calculate kendall's tau for sun and tmax        
    )


Now, let's group the data by season. First, a scatter plot colored by season:

In [ ]:
ggplot(data=station_data, mapping=aes(x=sun, y=tmax, color=season, shape=season)) + geom_point()

From this, it looks like the relationship is not consistent across seasons - there's not a clear relationship for winter, and autumn appears to be stronger than spring and summmer. We can check this by using `summarize()` to calculate the different correlation values using the output of `group_by()`:

In [ ]:
corr_table <- station_data |> 
    group_by(season) |> # group by season
    summarize(
        pearson = cor(sun, tmax, use='complete.obs'), # calculate pearson's r for sun and tmax
        spearman = cor(sun, tmax, use='complete.obs', method='spearman'), # calculate spearman's rho for sun and tmax
        kendall = cor(sun, tmax, use='complete.obs', method='kendall') # calculate kendall's tau for sun and tmax
    )

corr_table # show the table

## What is the relationship between `tmin` and `sun`? does it change by season?

We can proceed here in the same way that we did previously, just using `tmin` instead of `tmax`:

In [ ]:
ggplot(data=station_data, mapping=aes(x=sun, y=tmin)) + geom_point()

This still looks like a positive relationship, but less strong than the relationship for `tmax`. We can confirm this using `summarize()`:

In [ ]:
station_data |> 
    summarize(
        pearson = cor(sun, tmin, use='complete.obs'), # calculate pearson's r for sun and tmin
        spearman = cor(sun, tmin, use='complete.obs', method='spearman'), # calculate spearman's rho for sun and tmin
        kendall = cor(sun, tmin, use='complete.obs', method='kendall') # calculate kendall's tau for sun and tmin        
    )

In [ ]:
ggplot(data=station_data, mapping=aes(x=sun, y=tmin, color=season, shape=season)) + geom_point()

Interestingly, it looks like the relationship is still strongest for autumn, but we also have a strong(er), negative relationship for winter. Again, we can calculate a table of correlation values to compare:

In [ ]:
corr_table <- station_data |> 
    group_by(season) |> # group by season
    summarize(
        pearson = cor(sun, tmin, use='complete.obs'), # calculate pearson's r for sun and tmin
        spearman = cor(sun, tmin, use='complete.obs', method='spearman'), # calculate spearman's rho for sun and tmin
        kendall = cor(sun, tmin, use='complete.obs', method='kendall') # calculate kendall's tau for sun and tmin        
    )

corr_table # show the table

## Set up and fit a multiple linear regression model for `air_frost` as a function of `tmax`, `tmin`, `sun`, and `rain` in the winter. Which of these variables has the strongest effect on `air_frost`?

First, we can use `filter()` to select only winter observations:

In [ ]:
winter_obs <- station_data |> filter(season == 'winter')

Now, we can use `lm()` to create a linear model, using the **formula** `air_frost ~ tmax + tmin + sun + rain`:

In [ ]:
winter_mlm <- lm(air_frost ~ tmax + tmin + sun + rain, data=winter_obs)

To see the estimate, standard error, *t*-value, and *p*-value for each coefficient, we can use `summary()`:

In [ ]:
summary(winter_mlm)